# Code 4 - SQL Agents with Dynamic Few Shot Prompting

## Step 1 - Connect to Database

### Import Packages

In [1]:
from sqlite3 import connect
from langchain_community.utilities.sql_database import SQLDatabase

### Create database connection for LLM (create db)

In [2]:
db = SQLDatabase.from_uri("sqlite:///chinook.db", sample_rows_in_table_info = 3)
print(len(db.get_usable_table_names()), db.get_usable_table_names())

11 ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


### Check Connection

In [3]:
con = connect("chinook.db")
cur = con.execute("Select Count(Distinct(AlbumId)) from Album;")
print(cur.fetchall())

[(347,)]


## Step 2 - Setup Langchain SQL Chain

### Import API Key

In [4]:
#import os
#from dotenv import load_dotenv
#load_dotenv()

In [5]:
#API_KEY = os.getenv("OPENAI_API_KEY")

### Import Packages

In [6]:
from langchain_ollama.llms import OllamaLLM

### Create an Instance of LLM with configuration & Check (creating llm)

In [7]:
llm = OllamaLLM(model="llama3")
llm

OllamaLLM(model='llama3')

## Step 3 Setup Agent with Few Shot Examples

Advantages of SQL Agents:

1. Saves tokens - Retrieving only required schema & relevant tables.
2. Recover from errors by generating queries, catch the traceback and regenerate correctly
3. Chains run on a predefined pattern; Agents is defined by llm; The LLM then chooses the right tools to execute

### Create few Shot Examples

In [8]:
examples = [
    {   "input": "List all artists.", 
        "query": "SELECT * FROM Artist;"},
    {
        "input": "Find all albums for the artist 'AC/DC'.",
        "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
    },
    {
        "input": "List all tracks in the 'Rock' genre.",
        "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
    },
    {
        "input": "Find the total duration of all tracks.",
        "query": "SELECT SUM(Milliseconds) FROM Track;",
    },
    {
        "input": "List all customers from Canada.",
        "query": "SELECT * FROM Customer WHERE Country = 'Canada';",
    },
    {
        "input": "How many tracks are there in the album with ID 5?",
        "query": "SELECT COUNT(*) FROM Track WHERE AlbumId = 5;",
    },
    {
        "input": "Find the total number of Albums.",
        "query": "SELECT COUNT(DISTINT(AlbumId)) FROM Invoice;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "Employee"',
    },
]
print(len(examples))

10


### Import Embedding Packages

In [9]:
from langchain_community.embeddings import OllamaEmbeddings

### Create an Instance of Embeddings

In [10]:
embeddings = (
    OllamaEmbeddings(model = "llama3")
)

### Sample Embedding

In [11]:
len(embeddings.embed_query("Hi"))

4096

### Import Vector Store Packages

In [12]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

### Embed Examples

In [13]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=3,
    input_keys=["input"],
    )

### Check Similarity of Query in Vector DB

In [14]:
matched_queries = example_selector.vectorstore.search("How many arists are there?", search_type = "mmr")
print(matched_queries)

[Document(metadata={'input': 'How many tracks are there in the album with ID 5?', 'query': 'SELECT COUNT(*) FROM Track WHERE AlbumId = 5;'}, page_content='How many tracks are there in the album with ID 5?'), Document(metadata={'input': 'List all artists.', 'query': 'SELECT * FROM Artist;'}, page_content='List all artists.'), Document(metadata={'input': 'How many employees are there', 'query': 'SELECT COUNT(*) FROM "Employee"'}, page_content='How many employees are there'), Document(metadata={'input': 'Who are the top 5 customers by total purchase?', 'query': 'SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;'}, page_content='Who are the top 5 customers by total purchase?')]


In [15]:
for doc in matched_queries:
    print(doc.page_content)

How many tracks are there in the album with ID 5?
List all artists.
How many employees are there
Who are the top 5 customers by total purchase?


In [16]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool, InfoSQLDatabaseTool, ListSQLDatabaseTool, QuerySQLCheckerTool

In [17]:
sql_db_query =  QuerySQLDataBaseTool(db = db)
sql_db_schema =  InfoSQLDatabaseTool(db = db)
sql_db_list_tables =  ListSQLDatabaseTool(db = db)
sql_db_query_checker = QuerySQLCheckerTool(db = db, llm = llm)

In [18]:
tools = [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker]

In [19]:
for tool in tools:
    print(tool.name + " - " + tool.description.strip() + "\n")

sql_db_query - Execute a SQL query against the database and get back the result..
    If the query is not correct, an error message will be returned.
    If an error is returned, rewrite the query, check the query, and try again.

sql_db_schema - Get the schema and sample rows for the specified SQL tables.

sql_db_list_tables - Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker - Use this tool to double check if your query is correct before executing it.
    Always use this tool before executing a query with sql_db_query!



### Create System prefix Prompt

In [20]:
system_prefix = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Here are some examples of user inputs and their corresponding SQL queries:

"""

In [21]:
suffix = """
Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

### Import Template Packages

In [22]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate

In [23]:
dynamic_few_shot_prompt_template = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input"],
    prefix=system_prefix,
    suffix=suffix
)

### Integrate few Shot Prompts with User Inputs

In [24]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate

In [25]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=dynamic_few_shot_prompt_template),
    ]
)

#### Check Prompt

In [26]:
print(full_prompt.messages[0].to_json()['kwargs'])

{'prompt': FewShotPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001BB00049D90>, k=3, example_keys=None, input_keys=['input'], vectorstore_kwargs=None), example_prompt=PromptTemplate(input_variables=['input', 'query'], template='User input: {input}\nSQL query: {query}'), suffix='\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}\n', prefix='\nAnswer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answ

### Validate Prompt

In [27]:
prompt_val = full_prompt.invoke(
    {
        "input": "How many arists are there?",
        "tool_names" : [tool.name for tool in tools],
        "tools" : [tool.name + " - " + tool.description.strip() for tool in tools],
        "agent_scratchpad": [],
    }
)

print(prompt_val.to_string())

System: 
Answer the following questions as best you can. You have access to the following tools:

['sql_db_query - Execute a SQL query against the database and get back the result..\n    If the query is not correct, an error message will be returned.\n    If an error is returned, rewrite the query, check the query, and try again.', 'sql_db_schema - Get the schema and sample rows for the specified SQL tables.', 'sql_db_list_tables - Input is an empty string, output is a comma-separated list of tables in the database.', 'sql_db_query_checker - Use this tool to double check if your query is correct before executing it.\n    Always use this tool before executing a query with sql_db_query!']

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker']]
Action Input: the input to the action
Observation: the

In [28]:
from langchain.agents import AgentExecutor, create_react_agent
agent = create_react_agent(llm, tools, full_prompt)

In [29]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

### Execute LLM

In [30]:
agent_executor.invoke({"input": "Provide the number of customers with respect to each country."})



> Entering new AgentExecutor chain...
Action: sql_db_query
Action Input: "SELECT Country, COUNT(*) AS CustomerCount FROM Customer GROUP BY Country[('Argentina', 1), ('Australia', 1), ('Austria', 1), ('Belgium', 1), ('Brazil', 5), ('Canada', 8), ('Chile', 1), ('Czech Republic', 2), ('Denmark', 1), ('Finland', 1), ('France', 5), ('Germany', 4), ('Hungary', 1), ('India', 2), ('Ireland', 1), ('Italy', 1), ('Netherlands', 1), ('Norway', 1), ('Poland', 1), ('Portugal', 2), ('Spain', 1), ('Sweden', 1), ('USA', 13), ('United Kingdom', 3)]Thought: Now that I have the number of customers with respect to each country, I should check if the query is correct before proceeding.

Action: sql_db_query_checker
Action Input: The SQL query provided earlier: "SELECT Country, COUNT(*) AS CustomerCount FROM Customer GROUP BY CountryHere is the output:

SELECT Country, COUNT(*) AS CustomerCount FROM Customer GROUP BY CountryIt looks like we have a correct query!

Since the query was checked and found to be

{'input': 'Provide the number of customers with respect to each country.',
 'output': 'Agent stopped due to iteration limit or time limit.'}

### Execute LLM Agent with Debug

In [31]:
#import langchain
#langchain.debug = True

In [32]:
agent_executor.invoke({"input": "Provide the number of customers with respect to each country."})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Provide the number of customers with respect to each country."
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain

{'input': 'Provide the number of customers with respect to each country.',
 'output': "The number of customers with respect to each country is [(1, 'Argentina'), (1, 'Australia'), (1, 'Austria'), (1, 'Belgium'), (1, 'Brazil'), (1, 'Canada'), (1, 'Chile'), (1, 'Czech Republic'), (1, 'Denmark'), (1, 'Finland'), (1, 'France'), (1, 'Germany'), (1, 'Hungary'), (1, 'India'), (1, 'Ireland'), (1, 'Italy'), (1, 'Netherlands'), (1, 'Norway'), (1, 'Poland'), (1, 'Portugal'), (1, 'Spain'), (1, 'Sweden'), (1, 'USA'), (1, 'United Kingdom')]"}

# END - Next Topic Custom Agent